In [7]:
#imports
import boto3
from botocore.exceptions import ClientError #for debugging
from dotenv import load_dotenv #for env variables
import os

In [8]:
#load the variables/password/IDs from the .env file
load_dotenv()
#get the access and secret key 
access_key_id = os.getenv('ACCESS_KEY_ID')
secret_key = os.getenv('SECRET_ACCESS_KEY')

#declare bucket name and data folder name where data will be downloaded to
bucket_name = "mindex-data-analytics-code-challenge"
data_folder = "/Users/mady/mindex_data_analytics_challenge/data"

#print  key variables to verify that hold the correct values as what was provided
#print('access_key_id: ', access_key_id)
#print('secret_key: ', secret_key)


In [9]:

#create s3 client
s3 = boto3.client('s3', aws_access_key_id=access_key_id, aws_secret_access_key=secret_key, region_name='us-east-1')

#file names to download
files = ['bengals.csv', 'boyd_receiving.csv', 'chase_receiving.csv', 'higgins_receiving.csv']

#download each file
for file in files:
    try:
        s3.download_file(bucket_name, file, data_folder + file)
        print(file + " downloaded successfully")
    except Exception as e:
        print(file + "error: ", e)

 

bengals.csv downloaded successfully
boyd_receiving.csv downloaded successfully
chase_receiving.csv downloaded successfully
higgins_receiving.csv downloaded successfully
